Load Librairies

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import scipy

Use Latex Font

In [3]:
plt.rcParams.update({
    "text.usetex": True,                # Use LaTeX for text
    "font.family": "serif",             # Use 'serif' font
    "font.serif": ["Computer Modern"],  # LaTeX default font
    "axes.labelsize": 14,               # Font size for labels
    "font.size": 11,                    # General font size
    "legend.fontsize": 11,              # Font size for legend
    "xtick.labelsize": 10,              # Font size for x-axis
    "ytick.labelsize": 10               # Font size for y-axis
})

Use color friendly colors

In [4]:
plt.style.use('tableau-colorblind10')

# Data

In [5]:
def identify_drought(time_series: pd.Series,threshold:float):
    events_thresh = time_series<threshold
    raw_events = events_thresh.groupby(events_thresh.ne(events_thresh.shift()).cumsum()).cumsum()
    identified_events = raw_events[(raw_events.shift(1) < raw_events) & (raw_events.shift(-1) < raw_events)]
    identified_events.index = pd.Index([date - pd.Timedelta(identified_events.loc[date]-1, unit='h') for date in identified_events.index])
    return identified_events

In [6]:
def calculate_return_period(droughts:pd.DataFrame):
    droughts_sorted = droughts.sort_values('Duration')
    droughts_sorted['rank'] = scipy.stats.rankdata(droughts_sorted['Duration'])
    droughts_sorted['exceedance_probability'] = (len(droughts_sorted)-droughts_sorted['rank']+1)/(len(droughts_sorted)+1)
    droughts_sorted['return_period'] = 1./droughts_sorted['exceedance_probability']
    droughts_sorted['return_years'] = droughts_sorted['return_period']*44/len(droughts_sorted)
    return droughts_sorted

In [ ]:
df_wind = pd.read_csv('../Data/cf_wind_7923.csv',
                         index_col=0,
                         parse_dates=True
)

df_pv = pd.read_csv('../Data/cf_pv_7923.csv',
                         index_col=0,
                         parse_dates=True
)

df_wind_pv = pd.concat([df_wind, df_pv])
df_avg = df_wind.rolling('D', min_periods=24, center=True).mean()

In [ ]:
droughts_dict = {}

for column in df_avg:
    raw_droughts = identify_drought(df_avg[column], 0.05)
    droughts = raw_droughts[raw_droughts > 24]
    droughts_df = pd.DataFrame(droughts)
    droughts_df = droughts_df.set_axis(['Duration'], axis=1)
    droughts_dict[column] = droughts_df

In [ ]:
return_dict = {}
for key, value in droughts_dict.items():
    return_dict[key] = calculate_return_period(value)

# Plot

In [ ]:
def my_title(ax, title):
    ax.set_title(title, 
                 x=0.5, y=0.945,
                 transform=ax.transAxes,
                 fontsize=16, 
                 bbox=dict(facecolor='white', alpha=1)
    )

In [ ]:
def plot_lines(axs):
    axs[2].plot((0.03, 45), (5,5), '--k', alpha=0.6)
    axs[3].plot((0.03, 45), (5,5), '--k', alpha=0.6)

    axs[2].plot((0.57, 0.57), (0,5), '--', color=color_models[0])
    axs[3].plot((4, 4), (0,5), '--', color=color_models[0])

    axs[2].plot((1.4, 1.4), (0,5), '--', color=color_models[1])
    axs[3].plot((5.5, 5.5), (0,5), '--', color=color_models[1])

    axs[2].plot((1.15, 1.15), (0,5), '--', color=color_models[2])
    axs[3].plot((4.9, 4.9), (0,5), '--', color=color_models[2])

In [ ]:
label_name = ['Atlite', 'C3S-E G', 'C3S-E N']
title_name = ['a) Wind', 'b) PV', 'c) Current', 'd) Projected']
color_models = ['#E24A33', '#348ABD', '#988ED5']    # ggplot
color_models = ['#006BA4', '#FF800E', '#ABABAB']    # colorblind


In [ ]:
duration_ranges_hours = [24, 48, 72, 96, 120, 144, 168, 192 ,np.inf]
duration_ranges_days = [dur / 24 for dur in duration_ranges_hours]

xlabel = ['1-2','2-3','3-4','4-5','5-6','6-7', '7-8','8+', '']

In [ ]:
categories = {
    'onshorewind': ['onshorewind_atlite', 'onshorewind_c3se_gridded', 'onshorewind_c3se_national'],
    'pv': ['pv_atlite', 'pv_c3se_gridded', 'pv_c3se_national'],
    'combine_2023': ['combine_atlite_2023', 'combine_c3se_gridded_2023', 'combine_c3se_national_2023'],
    'combine_2030': ['combine_atlite_2030', 'combine_c3se_gridded_2030', 'combine_c3se_national_2030']
}

In [ ]:
fig, axs = plt.subplots(2, 2, 
                        sharex = True,
                        # sharey = True,
                        figsize = (9, 5)
                        )
axs = axs.flatten()

for idx, (category, datasets_in_category) in enumerate(categories.items()):
    ax = axs[idx]

    for i, name in enumerate(datasets_in_category):

        ax.scatter(return_dict[name]['return_years'], return_dict[name]['Duration']/24, 
                   s = 10,
                   marker = 'x',
                   color = color_models[i],
                   alpha = 0.8,
                   zorder = 3
        )

    if idx != 1:
        ax.set_ylim(0,16)
    ax.set_xscale('log')
    ax.set_xticks([7/365, 30/365, 91/365, 1, 2, 5, 10, 40])
    ax.set_xticklabels(['1w', '1m', '3m', '1y', '2y', '5y', '10y', '40y'])
    ax.grid(alpha=0.5, zorder=0)
    my_title(ax, title_name[idx])


plot_lines(axs)
fig.legend(labels = label_name,
           loc = 'upper center',
           ncol = 3,
           markerscale = 2
           )

fig.supxlabel('Return period')
fig.supylabel('RES drought duration (days)')

fig.tight_layout()
fig.subplots_adjust(top=0.9)

# fig.savefig('../Figures/drouhgts_return_periods.pdf', dpi=300)